In [1]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import sys
from datetime import timedelta  

if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
from IPython.display import HTML
def View(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))    
    
print("Current Working directory " , os.getcwd())

Current Working directory  C:\Users\User\GitHub\WebET_Analysis


In [2]:
data_et = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_et.csv')
data_et_fix = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_et_fix.csv')
data_et_choice = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_et_choice.csv')

data_trial = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_trial.csv')
data_trial_fix = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_trial_fix.csv')
data_trial_choice = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_trial_choice.csv')

data_subject = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_subject.csv') \
    .rename(columns=
            {
                'chosenAmount': 'bonus_pay',
                'chosenDelay': 'bonus_delay'
            }
           )    

overview = pd.DataFrame(
    [
        [len(data_et)],
        [len(data_et_fix)],
        [len(data_et_choice)],
        [len(data_trial)],
        [len(data_trial_fix)],
        [len(data_trial_choice)],
        [len(data_subject)]
    ], 
    columns=['length'],
    index=['data_et', 'data_et_fix', 'data_et_choice', 
           'data_trial', 'data_trial_fix', 'data_trial_choice', 
           'data_subject'])
print(overview)

                   length
data_et            618566
data_et_fix         63432
data_et_choice     141719
data_trial          30362
data_trial_fix        929
data_trial_choice    3714
data_subject           84


# Approve subjects

In [3]:
if 'trial_length' in data_subject.columns: 
    print('trial_length already added!')
else:
    output = []
    for subject in data_trial['run_id'].unique():
        prolificID = data_trial.loc[data_trial['run_id']==subject, 'prolificID'].unique()[0]
        trial_length = len(data_trial.loc[data_trial['run_id']==subject, 'trial_index'].unique())
        output.append([subject, prolificID, trial_length])
    output = pd.DataFrame(output,
                 columns=['run_id', 'prolificID', 'trial_length']
                )

    data_subject = data_subject.merge(output, on=['run_id', 'prolificID'], how='left')

In [4]:
data_et = data_et.merge(
    data_trial.loc[:, ['run_id', 'chinFirst', 'trial_index', 'trial_type', 'task_nr']], 
    on=['run_id', 'trial_index'], 
    how='left'
)

In [5]:
output = []
for subject in data_et['run_id'].unique():
    m_x_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        ~((data_et['chinFirst']==1) & (data_et['task_nr']==2) |
          (data_et['chinFirst']==0) & (data_et['task_nr']==3) 
         ), 
        'x'
    ].mean()
    m_x_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'x'
    ].mean()

    m_y_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        ~((data_et['chinFirst']==1) & (data_et['task_nr']==2) |
          (data_et['chinFirst']==0) & (data_et['task_nr']==3) 
         ), 
        'y'
    ].mean()
    m_y_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'y'
    ].mean()

    m_count_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        ~((data_et['chinFirst']==1) & (data_et['task_nr']==2) |
          (data_et['chinFirst']==0) & (data_et['task_nr']==3) 
         ), 
        'x'
    ].count()

    m_count_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'x'
    ].count()
    
    m_choseTop = data_trial_choice.loc[data_trial_choice['run_id']==subject,
                                       'choseTop'].mean()

    output.append([subject, m_x_fix, m_y_fix, m_x_choice, m_y_choice, 
                   m_count_fix, m_count_choice, 
                   m_choseTop])

output = pd.DataFrame(output, 
                      columns=['run_id', 'm_x_fix', 'm_y_fix', 'm_x_choice', 'm_y_choice', 
                               'm_count_fix', 'm_count_choice',
                               'm_choseTop'])
output

data_subject = data_subject.merge(output, on='run_id', how='left')

## These subjects await review

In [6]:
data_subject.loc[data_subject['status']=='AWAITING REVIEW', 
                 ['run_id', 'prolificID', 'trial_length',
                 'm_x_fix', 'm_y_fix', 'm_x_choice', 'm_y_choice', 'm_count_fix', 'm_count_choice',
                 'm_choseTop']
                ].sort_values(by='prolificID')

,run_id,prolificID,trial_length,m_x_fix,m_y_fix,m_x_choice,m_y_choice,m_count_fix,m_count_choice,m_choseTop


In [7]:
if 'choice_rt' in data_subject.columns:
    print('choice_rt already added!')
else: 
    grouped = data_trial.loc[data_trial['trial_type']=='eyetracking-choice', :] \
        .groupby(['prolificID'])['trial_duration_exact'].mean() \
        .reset_index() \
        .rename(columns={'trial_duration_exact': 'choice_rt'})
    data_subject = data_subject.merge(grouped, on='prolificID', how='left')

data_subject.loc[
    data_subject['status']=='AWAITING REVIEW', 
    ['run_id', 'prolificID', 'webcam_fps', 'choice_rt', 'bonus_pay', 'bonus_delay']
] \
    .sort_values(by='run_id')

KeyError: 'Column not found: trial_duration_exact'

# Basic payment

In [ ]:
data_pay = data_subject.loc[data_subject['status']=='APPROVED', 
                            [
                                'run_id', 'prolificID', 
                                'age',  'Country of Birth', 'Current Country of Residence', 'First Language',
                                'Nationality', 'Sex',
                                'status', 'reviewed_at_datetime', 'Country of Birth', 'entered_code',
                                'session_id', 'started_datetime', 'completed_date_time', 'time_taken',
                                'bonus_pay', 'bonus_delay'
                            ]
                           ]

data_pay['basic_pay'] = 2.25

# Bonus payment

Reformat payments

In [ ]:
def choice_options_to_numeric(data, varName): 
    data.loc[: , [varName]] = \
        data.loc[:, [varName]] \
            .replace(['Today', 'Tomorrow', '7 days', 
                      '15 days', '30 days', '90 days', 
                      '180 days'], 
                     [0, 1, 7, 15, 30, 90, 180]) \
            .replace({'\$':''}, regex = True) \
            .replace('50 cent', 0.5) 
    return data

for var in ['bonus_pay', 'bonus_delay']:
    data_pay = choice_options_to_numeric(data_pay, var)

data_pay.loc[: , ['bonus_pay', 'bonus_delay']]

In [ ]:
data_pay[['run_id', 'prolificID', 'bonus_pay']]

## Replace empty values with the best bonus reward

In [ ]:
# For two subjects, the bonus payment was not recorded
data_pay.loc[
    data_pay['prolificID'].isin(
        ['5fea6632bf9ae4a79153efdf', 
         '5b8969006651ea000118e42e', 
         '5fb2af792942a58ffe303948', 
        ]
    ), ['bonus_pay', 'bonus_delay']] = [5, 1]

data_pay.loc[data_pay[['bonus_pay', 'bonus_delay']].isnull().any(axis=1), :]

## What bonus is due today? 

In [ ]:
def add_completed_date(data, data_trial):
    output = []

    for subject in data_trial['run_id'].unique():
        thisSubject = data_trial.loc[data_trial['run_id']==subject] \
            .reset_index(drop=True)
        date_time_obj = datetime.datetime.strptime(
            thisSubject.loc[0, 'recorded_at'], '%Y-%m-%d %H:%M:%S')

        output.append([thisSubject.loc[0, 'run_id'], date_time_obj.date()])
        
    output = pd.DataFrame(output, columns=['run_id', 'date']) 
    
    if 'date' in data.columns: data = data.drop(columns=['date'])
    data = data.merge(output, on='run_id', how='left')
    return data

data_pay = add_completed_date(data_pay, data_trial)
    
data_pay['completed_date']=data_pay.apply(
    lambda x: datetime.datetime.strptime(
        x['completed_date_time'], '%Y-%m-%d %H:%M:%S.%f') \
            .date(),
    axis=1)
data_pay['bonus_pay'].unique()

In [ ]:
data_pay['reward_pound'] = data_pay['bonus_pay'] * 0.75

In [ ]:
data_pay['due_on'] = data_pay['completed_date'] +  data_pay['bonus_delay'].map(datetime.timedelta) 

data_pay.loc[:, 
               [
                   'prolificID', 'run_id',
                   'Nationality', 'Sex',  
                   'completed_date', 'bonus_pay', 'bonus_delay', 
                   'reward_pound', 'due_on'
               ]
              ]. sort_values(by='due_on')

In [ ]:
bonus_due_today = data_pay.loc[data_pay['due_on']==datetime.datetime.now().date(), :]
bonus_due_today['reward_pound'] = bonus_due_today['reward_pound'].round(2)
bonus_due_today

In [ ]:
bonus_due_today.loc[: , ['prolificID', 'reward_pound']] \
    .to_csv('C:/Users/User/Google Drive/Masterarbeit/Acquise/bonusToday.csv', 
            index=False, 
            header=False
           )
print(bonus_due_today['due_on'].unique())

# Total costs

In [ ]:
data_pay['basic_cost'] = data_pay['basic_pay'] * 1.4
data_pay['total_cost'] = (data_pay['basic_pay'] + data_pay['bonus_pay']) * 1.4
print('Basic Costs: ' + str(sum(data_pay['basic_cost'])))
print('Total Costs: ' + str(sum(data_pay['total_cost'])))
data_pay

# Export modified prolific data

In [ ]:
data_pay.to_csv('C:/Users/User/Google Drive/Masterarbeit/Acquise/data_pay.csv', index=False, header=True)

# Feedback

In [ ]:
print('Success! Script ran through.')